In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import math
import fastavro
import boto3
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from sklearn.model_selection import train_test_split
import re
import ast
import geotable
# in order to install gdal first you have to go that link https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal download the version compatible with your
# PC and then pip install GDAL-x.x.x-cp39-cp39-win_amd64.whl on your terminal or cmd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from difflib import SequenceMatcher
import psycopg2
from sqlalchemy import create_engine

In [ ]:
def otherz_zone(df):
    res = df
    indcies = res.loc[res['zone_id'] == 113].index
    for i in indcies:
        if res['zone_name'].loc[i] == 'others':
            res.at[i, 'zone_id'] = 136
    return res

In [ ]:
def similar(a, b):
    '''
    input:
        a: string
        b: string
    returns:
         percentage of how much a matches b 
    '''
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
def match(series,karims):
    '''
    input:
        series: is a pandas series of zone names
        karims: is a pandas dataframe that consists of {zone_id:int, zone_name_en: string, zone_name_ar}
    returns:
        list of ids in the same order of series 
    '''
    dict_ids = {}
    for i in series.unique():
        perc = karims['en'].apply(lambda x: similar(x,i))
        maxm = perc.max()
        indx = perc.loc[perc == maxm].index[0]
        Id = karims.loc[indx]['id']
        dict_ids[i] = Id
    df1 = pd.DataFrame({'zone_name' : dict_ids.keys(), 'zone_id' : dict_ids.values()})
    #res = pd.merge(users_zones['zone_name'],df1, how = 'left',on = 'zone_name')
    return df1

In [ ]:
def getpolys(ser,long,lat):
    point = Point(long,lat)
    c = 0
    for i in ser:
        if i.contains(point) or i.touches(point):
            return c
        else:
            c+=1
    return -1

In [ ]:
def zoneser(ser_poly,ser_long,ser_lat):
    """"inputs: 
             ser_poly: is a pandas series that contains the polygons of each zone
             ser_long: is a pandas series that contains the longitudes 
             ser_long: is a pandas series that contains the latitude
        returns:
                points: a pandas series that contains the indecies of the zones 
    """
    points = pd.DataFrame({"long":ser_long,"lat":ser_lat})
    #points['points'] = points.apply(lambda x: Point(x.long, x.lat), axis=1)
    points['zone_index'] = points.apply(lambda x : getpolys(ser_poly,x.long,x.lat), axis=1)
    return points
t = geotable.load('D:\zones mapping files\AL 3 Zones 2022-08-31.kmz')
t.head()

In [ ]:
def getLastCreatedat(series):
    date_time = datetime.fromtimestamp(series.values[-1])  
    return str(date_time)

In [ ]:
karims = pd.read_excel('D:\zones mapping files\\karims2.xlsx')#,names=['id','en','ar'])
print(karims.shape)
karims.columns = ['id','en','ar']
new_data = pd.DataFrame({'id': [1], 'en': ['Al Baha'], 'ar': ['الباحة']})
karims = pd.concat([karims, new_data], ignore_index=True)
print(karims.shape)
karims.head()

In [ ]:
#query to get my interactions data

In [ ]:
%%time
#inters_update = pd.read_csv('D:\interactions_only\interactions_only_update\\raw data\interaction.csv')
inters_update.rename(columns={'DropOff_Longitude' : 'long','DropOff_Latitude':'lat'}, inplace = True)
inters_update = inters_update.drop_duplicates()
print(inters_update.shape)
print(inters_update.dtypes)
inters_update.tail()

In [ ]:
%%time
zone_df = zoneser(t.geometry_object, inters_update.long, inters_update.lat)
print(zone_df.shape)
print(zone_df.dtypes)
zone_df.head()

In [ ]:
%%time
zone_df['zone_name'] = zone_df.apply(lambda x: t['Name'][x.zone_index] if x.zone_index != -1  else "others", axis=1)
zone_df = zone_df.drop_duplicates()
print(zone_df.shape)
print(zone_df.dtypes)
zone_df.head()

In [ ]:
merged_locs = pd.merge(left=inters_update, right=zone_df, how="left", on=['long','lat'])
merged_locs = merged_locs.drop_duplicates()
# merged_locs = pd.merge(inters_update,zone_df)
merged_locs.rename(columns={'OrderStatus' : 'event_value'}, inplace = True)
merged_locs['event_value'] = merged_locs['event_value'].values.astype(float)
merged_locs['TIMESTAMP'] = merged_locs['TIMESTAMP'].values.astype('long')
print(merged_locs.shape)
print(merged_locs.dtypes)
merged_locs.tail()

In [ ]:
%%time
merged_locs = pd.merge(merged_locs,match(merged_locs['zone_name'],karims),how='left',on='zone_name')

In [ ]:
date_time = datetime.fromtimestamp(merged_locs.loc[merged_locs['zone_name'] == 'رياض الخبراء / Riyadh Al Khabra']['TIMESTAMP'].values[0])  
date_time

In [ ]:
merged_locs = merged_locs[['USER_ID','ITEM_ID','TIMESTAMP', 'zone_name', 'zone_id']]
merged_locs.head()

In [ ]:
otherz_zone(merged_locs)

In [ ]:
merged = pd.concat([merged_locs, interactions], ignore_index=True)

In [ ]:
merged = merged.drop_duplicates()

### Users update

In [ ]:
#query to get my Users data

In [ ]:
#users_updated = pd.read_csv('D:\interactions_only\interactions_only_update\\raw data\\users.csv')
users_updated.rename(columns={'DropOff_Longitude' : 'long','DropOff_Latitude':'lat'}, inplace = True)
users_updated = users_updated.drop_duplicates()
print(users_updated.shape)
print(users_updated.dtypes)
print(users_updated['BuyerId'].value_counts())
users_updated.head()

In [ ]:
# mapping users locations with the zones
users_zone_df = zoneser(t.geometry_object, users_updated.long, users_updated.lat)
print(users_zone_df.shape)
print(users_zone_df.dtypes)
users_zone_df.head()

In [ ]:
# adding the zone name to the mapping using indicies
users_zone_df['zone_name'] = users_zone_df.apply(lambda x: t['Name'][x.zone_index] if x.zone_index != -1  else "others", axis=1)
users_zone_df = users_zone_df.drop_duplicates()
print(users_zone_df.shape)
print(users_zone_df.dtypes)
users_zone_df.head()

In [ ]:
users_zones = pd.merge(left=users_updated, right=users_zone_df, how="left", on=['long','lat'])
users_zones = users_zones.drop_duplicates()
print(users_zones.shape)
print(users_zones.dtypes)
users_zones.head()

In [ ]:
#users_updated = users_updated[['BuyerId','harvesine_distance']]
users_zones = users_zones[['BuyerId','zone_name','zone_index']]
users_zones.rename(columns = {'BuyerId':'USER_ID'}, inplace = True)
users_zones.head()

In [ ]:
%%time
users_zones = pd.merge(users_zones,match(users_zones['zone_name'],karims),how='left',on='zone_name')

In [ ]:
u = otherz_zone(users_zones)

### Stores

In [ ]:
#query to get my Stores data

In [ ]:
#stores = pd.read_csv('D:\interactions_only\interactions_only_update\\raw data\\stores.csv')
stores.drop_duplicates()
stores.rename(columns={'Location_Longitude' : 'long','Location_Latitude':'lat'}, inplace = True)
print(stores.shape)
print(stores.dtypes)
stores.head()

In [ ]:
#mapping stores locations to its zones
stores_zone_df = zoneser(t.geometry_object, stores.long, stores.lat)
print(stores_zone_df.shape)
print(stores_zone_df.dtypes)
stores_zone_df.head()

In [ ]:
# adding the zone name to the mapping using indicies
stores_zone_df['zone_name'] = stores_zone_df.apply(lambda x: t['Name'][x.zone_index] if x.zone_index != -1  else "others", axis=1)
stores_zone_df = stores_zone_df.drop_duplicates()
print(stores_zone_df.shape)
print(stores_zone_df.dtypes)
stores_zone_df.head()

In [ ]:
stores_zones = pd.merge(left=stores, right=stores_zone_df, how="left", on=['long','lat'])
stores_zones = stores_zones.drop_duplicates()
print(stores_zones.shape)
print(stores_zones.dtypes)
stores_zones.head()

In [ ]:
stores_zones.rename(columns = {'Id':'ITEM_ID','StoreCategoryTypeId':'CATEGORY_L1'},inplace = True)

In [ ]:
#b7awl akhleha oryba mn el users tabel
stores_zones = stores_zones[['ITEM_ID','zone_name','zone_index']]

In [ ]:
%%time
stores_zones = pd.merge(stores_zones,match(stores_zones['zone_name'],karims),how='left',on='zone_name')

In [ ]:
s = otherz_zone(stores_zones)

In [ ]:
s = s[['ITEM_ID','zone_name','zone_id']]
print(s.shape)
s.head()

### view latest preprocessed data

### Stores

In [ ]:
stores_zones = pd.read_csv('D:\interactions_only\interactions_only_update\\updates4\\stores_after_update23.csv', index_col = False)
stores_zones = stores_zones.drop_duplicates()
print(stores_zones.shape)
print(stores_zones.dtypes)
stores_zones.head()

### Users

In [ ]:
users_zones = pd.read_csv('D:\interactions_only\interactions_only_update\\updates4\\users_after_update23.csv', index_col = False)
print(users_zones.shape)
print(users_zones.dtypes)
users_zones.head(10)

### Interactions

In [ ]:
interactions = pd.read_csv('D:\interactions_only\interactions_only_update\\updates4\interactions_after_update23.csv')
print(interactions.shape)
print(interactions.dtypes)
interactions.tail()

In [ ]:
import os
files = ['interactions_after_update2','users_after_update2','stores_after_update2']

for i in files:

    file_to_delete = 'D:\interactions_only\interactions_only_update\\updates4\{}.csv'.format(i)

    try:
        os.remove(file_to_delete)
        print(f"{file_to_delete} deleted successfully.")
    except FileNotFoundError:
        print(f"{file_to_delete} not found.")
    except Exception as e:
        print(f"Error deleting {file_to_delete}: {e}")


In [ ]:

files_to_rename = ['interactions_after_update23','users_after_update23','stores_after_update23']
for i in range(len(files)):
    old_file_name = 'D:\interactions_only\interactions_only_update\\updates4\{}.csv'.format(files_to_rename[i])
    new_file_name = 'D:\interactions_only\interactions_only_update\\updates4\{}.csv'.format(files[i])

    try:
        os.rename(old_file_name, new_file_name)
        print(f"{old_file_name} renamed to {new_file_name} successfully.")
    except FileNotFoundError:
        print(f"{old_file_name} not found.")
    except Exception as e:
        print(f"Error renaming {old_file_name} to {new_file_name}: {e}")